In [0]:
%sql
MERGE INTO projetos.silver.ecommerce_transactions AS target
USING (                              
  WITH treated_base AS (
    SELECT
      event_date,
      to_timestamp(CONCAT(event_date,' ',event_time)) AS event_timestamp,
      invoice_no,
      stock_code,
      description,
      quantity,
      unit_price,
      customer_id,
      country,
      CURRENT_TIMESTAMP() AS ingestion_timestamp,
      MD5(CONCAT(invoice_no, stock_code, quantity, unit_price)) AS hash,
      'ecommerce_sales.csv' AS data_source, 
      REPLACE(REPLACE(CONCAT('run_', CURRENT_DATE()), ' ', ''), '-', '') AS pipeline_run_id 
    FROM (
      SELECT
        to_date(REPLACE(substring_index(InvoiceDate,' ',1), '/', '-'),'M-d-yyyy') AS event_date,
        CASE
          WHEN LENGTH(substring_index(substring_index(InvoiceDate,' ', -1), ':', 1)) = 1
          THEN CONCAT('0',
                      substring_index(substring_index(InvoiceDate,' ', -1), ':', 1),
                      ':',
                      substring_index(InvoiceDate,':', -1))
          ELSE substring_index(InvoiceDate,' ', -1)
        END                                     AS event_time,
        InvoiceNo   AS invoice_no,
        StockCode   AS stock_code,
        Description AS description,
        Quantity    AS quantity,
        UnitPrice   AS unit_price,
        CustomerID  AS customer_id,
        Country     AS country
      FROM projetos.bronze.ecommerce_transactions
      WHERE StockCode REGEXP '^[^A-Z]' AND Quantity > 0 AND UnitPrice > 0
    )
  )
  SELECT * FROM treated_base
) AS source
ON  target.event_date      = source.event_date
AND target.event_timestamp = source.event_timestamp
AND target.invoice_no      = source.invoice_no
AND target.stock_code      = source.stock_code
AND target.description     = source.description
AND target.quantity        = source.quantity
AND target.unit_price      = source.unit_price
AND target.customer_id     = source.customer_id
AND target.country         = source.country
AND target.hash            = source.hash
WHEN NOT MATCHED THEN
  INSERT (event_date, event_timestamp, invoice_no, stock_code, description, quantity, unit_price, customer_id, country, ingestion_timestamp, hash, data_source, pipeline_run_id)
  VALUES (source.event_date, source.event_timestamp, source.invoice_no, source.stock_code, source.description, source.quantity, source.unit_price, source.customer_id, source.country, source.ingestion_timestamp, source.hash, source.data_source, source.pipeline_run_id)

In [0]:
%sql
OPTIMIZE projetos.silver.ecommerce_transactions;

In [0]:
%sql
ALTER TABLE projetos.silver.ecommerce_transactions SET TAGS (
  'last_run' = '${runtime}'
)